In [18]:
import math
import random
import pyspark
from itertools import chain
import datetime
import os
import time
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

global k_fold
global n_row
global y_h_m_y
global n_columns
global w
global OUTPUT
global ITERATIONS
global THRESHOLD
global averages
global col_sums
global sigmas
global b

DEBUG = True
TRAIN_TEST = 0.8
k_fold=3
ITERATIONS=50
THRESHOLD = 0.5

DATASET="dataset/spam.data"
n_columns=56 #total number of columns (58) - the one deleted (57) - label (56)

OUTPUT="out/"

def initializeAccumulators():
    global averages
    global col_sums
    global sigmas
    i=0
    averages=[]
    col_sums=[]
    sigmas=[]
    while(i<n_columns):
        averages.append(sc.accumulator(0))
        col_sums.append(sc.accumulator(0))
        sigmas.append(sc.accumulator(0)) 
        i+=1

def addToAccumulators(row):
    if(len(row)!=len(col_sums)):
        raise Exception("Number of columns in the row doesn't mach the number of accumulators initiated. Len row: "+str(len(row))+" n_accomulators: "+str(len(col_sums)))
    i=0
    while(i<n_columns):
        col_sums[i].add(row[i])
        i+=1
    #print("Col sum len: " + str(len(col_sums)))
        
def preprocessing(row):
    split=row.split(" ")       #splitting
    label = int(split[57])
    del split[57] #label
    del split[56] #col 57th
    split=[float(col) for col in split]
    addToAccumulators(split)
    #assign random key to the datapoint
    key=random.getrandbits(64)
    #assign train/test 
    if(random.random()<TRAIN_TEST):
        train=1
    else:
        train=0    
    return (key,train,split,label)
def calcAvg(n_row):
    global averages
    i=0
    while(i<n_columns):
        averages[i]=col_sums[i].value/n_row
        i+=1
def calcResiduals(row):
    global sigmas
    i=0
    while(i<n_columns):       
        sigmas[i].add(math.pow(row[2][i]-averages[i],2))
        i+=1
    return row
def calcSigmas(n_row):
    global sigmas
    i=0
    while(i<n_columns):
        sigmas[i]=math.sqrt(sigmas[i].value/float(n_row-1))
        i+=1

def normalize(row):
    i=0
    while(i<n_columns):
        row[2][i]=(row[2][i]-averages[i])/sigmas[i]
        i+=1
    return row

def initializeWeights(random_init=False):
    if(random_init):
        #return sc.parallelize([(i, random.random()) for i in range(0,n_columns)])
        return [random.random() for i in range(0,n_columns)]
    else:
        #return sc.parallelize([(i, 0.0) for i in range(0,n_columns)])
        return [0.0 for i in range(0,n_columns)]

def initializeBias():
    return sc.parallelize(0.0)

def sigmoid(z):
    return 1/(1+math.exp(-z))

def predict(w,b,X):
    sig=sigmoid(sum([X[i]*w[i] for i in range(len(w))])+b)
    #print("Sig: "+str(sig)+"w: "+str(w)+"b: "+str(b)+"X: "+str(X))
    return sig

def classify_prediction(pred_probability):
    if (pred_probability >= THRESHOLD):
        return 1
    return 0

def predict_parallel(w,b,X):
    #Change X representation
    X = sc.parallelize(X).zipWithIndex().map(lambda x: (x[1],x[0]))
    wX_plus_b=X.join(w).map(lambda x: (x[1][0]*x[1][1])+b).sum()
    return sigmoid(wX_plus_b)

def compute_cost(dataset,w,b,lambda_reg, print_stats=False):
    cost=0
    #print("predict(w,b,x[2]))="+str(predict(w,b,dataset.collect()[0][2])))
    #print("math.log(predict(w,b,x[2]))="+str(math.log(predict(w,b,dataset.collect()[0][2]))))
    cost = (-1/dataset.count())*dataset.map(lambda x: x[3]*math.log(predict(w,b,x[2]))+(1-x[3])*math.log(1-predict(w,b,x[2]))).sum()+lambda_reg/(2*dataset.count())*sum([i*i for i in w])
    
    '''
    print("RowNumber: "+str(dataset.count()))
    print("\nlambda_reg: "+str(lambda_reg))
    print("\nb: "+str(b))
    print("\nw: "+str(w))
    print("\nDataset: "+str(w))
    '''
    if (print_stats):
        stats = {"TP":0, "TN":0, "FP":0, "FN":0}
        statsMapping = dataset.map(lambda x: 2*(classify_prediction(predict(w, b, x[2])) - x[3]) + x[3])  #this function maps all 4 possible variations to an integer, from -1 to 3
        stats["TP"] = statsMapping.filter(lambda x: x == 1).count()
        stats["TN"] = statsMapping.filter(lambda x: x == 2).count()
        stats["FP"] = statsMapping.filter(lambda x: x == -1).count()
        stats["FN"] = statsMapping.filter(lambda x: x == 0).count()
        precision = stats["TP"]/(stats["TP"] + stats["FP"])
        recall = stats["TP"]/(stats["TP"] + stats["FN"])
        accuracy = (stats["TP"] + stats["TN"])/(stats["TP"] + stats["TN"] + stats["FP"] + stats["FN"])
        print("Precision: " + str(precision))
        print("Recall: " + str(recall))
        print("F1 score: " + str(2*(precision*recall)/(precision + recall)))
        print("Accuracy: " + str(accuracy))
    return cost

def make_folds(x):
    return (x[0],int(random.random()*10-1),x[1],x[2])


def touch(path):
    with open(path, 'w+'):
        os.utime(path, None)

def gradientDescent(iterations,train,w,number_samples,lambda_reg,learning_rate,b,logCosts=False):
    if(logCosts):
        gd_cost_log = open(OUTPUT+"gradient_descent.csv","w+")
        gd_cost_log.write("iteration,cost\r\n")
        gd_cost_log.close()
    costs=[]
    for iteration in range(iterations):
        w,b = gradientDescentIteration(train,w,number_samples,lambda_reg,learning_rate,b)
        #print("b: "+str(b))
        #print(str(w))
        
        cost=compute_cost(train,w,b,lambda_reg)
        #print("cost:"+str(cost))
        costs.append(cost)
        if(logCosts):
            gd_cost_log = open(OUTPUT+"gradient_descent.csv","a+")            
            gd_cost_log.write(str(iteration+1)+","+str(cost)+"\r\n")
            gd_cost_log.close()
        print("-> Iteration done: "+str(iteration+1)+" of "+str(iterations)+". Cost: "+str(cost))
    return w,b,costs
    
global flag
flag=True
    
def gradientDescentIteration(train,w,number_samples,lambda_reg,learning_rate,b):
    global flag
    dw=[0 for i in range(0,n_columns)]
    j = 0
    w_temp = w.copy()
    for j in range(n_columns):
        X_j=train.map(lambda x: (predict(w,b,x[2])-x[3])*x[2][j]).sum()
        #print("X_j"+str(X_j)+"---")
        dw[j]=(1/number_samples)*X_j+(lambda_reg/number_samples)+w[j]
        #print("w[j] "+str(w[j]))
        w_temp[j]-=learning_rate*dw[j]
        #print(str(w[j]))
    b-=learning_rate*(1/number_samples)*train.map(lambda x: predict(w,b,x[2])-x[3]).sum()
    w = w_temp
    return w_temp,b


def kFoldsCV(k_fold,iterations,train,lambda_reg,learning_rate):
    fold_length=train.count()/k_fold
    train_errors_fold=[]
    test_errors_fold = []
    for i_fold in range(k_fold):
        w=initializeWeights()
        b=0
        starting_fold=fold_length*i_fold
        end_fold=starting_fold+fold_length
        test_fold=train.zipWithIndex().filter(lambda t: (t[1]>=starting_fold and t[1]<end_fold)).map(lambda t: t[0]) #the map get rid of the index again
        train_fold=train.zipWithIndex().filter(lambda t: t[1]<starting_fold or t[1]>=end_fold).map(lambda t: t[0])
        train_fold_size=train_fold.count()
        
        #Gradient descent
        w, b, train_errors = gradientDescent(iterations,train_fold,w,train_fold_size,lambda_reg,learning_rate,b)
        train_errors_fold.append(train_errors)
        train_errors_flattened =list(chain.from_iterable(train_errors_fold))
        test_error = compute_cost(test_fold, w, b, lambda_reg)
        test_errors_fold.append(test_error)
        if(DEBUG):
            print("--> Fold #"+str(i_fold+1)+" of "+str(k_fold)+" is done. Train error: " \
                  +str(sum(train_errors_flattened)/((i_fold + 1) * iterations)) \
                  + " Test error: " + str(sum(test_errors_fold)/(i_fold+ 1)))
    return w,b,train_errors_fold, test_errors_fold


def trainTestSplit(dataset):
    dataset=dataset.map(normalize)
    train=dataset.filter(lambda x: x[1]==1)
    #train_size=train.count()
    test=dataset.filter(lambda x: x[1]==0)
    return train, test

def testParameters(test, w, b):
    test_error = test.map(lambda x: ())

def train(filename, iterations, learning_rate, lambda_reg, cv=False):
    global w
    global b
    w = []
    initializeAccumulators()
    w = initializeWeights()
    dataset=sc.textFile(filename).map(preprocessing)
    n_row=dataset.count()
    calcAvg(n_row)
    gg = dataset.map(calcResiduals).collect()
    dataset=dataset.collect()
    calcSigmas(n_row)
    dataset=sc.parallelize(sc.parallelize(dataset).take(1))#.sortBy(lambda x: x[0])
    #dataset=sc.parallelize(dataset).sortBy(lambda x: x[0])
    n_row=dataset.count()
    '''
    print("AVg"+str(averages))
    print("Sigmas"+str(sigmas))
    print("First:"+str(train.take(1)))
    '''
    train, test = trainTestSplit(dataset)
    print("Split train/test done. Train contains "+str(train.count())+" elements, Test contains "+str(test.count())+" elements")
    
    if (cv):
        w, b, train_errors_fold, test_errors_fold = kFoldsCV(k_fold,iterations,train,lambda_reg,learning_rate)
        train_errors_fold =list(chain.from_iterable(train_errors_fold))
        average_train_error = sum(train_errors_fold)/(iterations*k_fold)
        average_test_error = sum(test_errors_fold)/k_fold
        print("---> "+str(k_fold)+"-fold validation done.  Train error: " \
            +str(average_train_error) \
            + " Test error: " + str((average_test_error)))
        return average_train_error, average_test_error
    else:                                                  #Testing the parameters and weights on the actual test set
        w,b, train_errors = gradientDescent(iterations, train, w, train.count(), lambda_reg, learning_rate, b,True)
        wBest = open(OUTPUT+"w_bestModel.csv","w+")
        wBest.write(str(w))
        wBest.close()
        average_train_error = sum(train_errors)/iterations
        average_test_error = compute_cost(test, w, b, lambda_reg, True)
        print("---> Model done. Train error: " \
              +str(average_train_error) \
              + " Test error: " + str((average_test_error)))
        return average_train_error, average_test_error
            

        
workers = range(8,9) #8 cores

working_time = open(OUTPUT+"working_times.csv","w+")
working_time.write("workers,grid_time,best_model_time,total\r\n")
working_time.close()
grid_log = open(OUTPUT+"grid.csv","w+")
grid_log.write("learning_rate,lambda_red,train_error,test_error\r\n")
grid_log.close()
for worker in workers:
    global b
    #working_time = open(OUTPUT+"working_times.csv","a+")
    print("****Worker: "+str(worker)+"****\n")
    sc = pyspark.SparkContext.getOrCreate()
    #sc.stop()
    #conf = (SparkConf().set("spark.cores.max", str(worker)))
    #sc = SparkContext(conf=conf)
    b = 0
    col_sums=[]
    averages=[]
    sigmas=[]
    w=[]
    if (False):
        grid=[]
        for i in range(1,5):
                for j in range(1,14,3):
                    grid+=[(i*0.09,j*0.028)]

        #grid=[(0.01,)] #(learning_rate,lambda_reg)
        #grid=list(chain.from_iterable(grid))
        grid_results = []
        #workers_executioTime_grid=time.time()
        for par in grid:
            print("|---- Starting training for parameters (learning_rate,lambda_reg) = "+str(par)+" ----|")
            result=(par, train(DATASET,ITERATIONS,par[0],par[1], True))
            grid_results.append(result)
            grid_log = open(OUTPUT+"grid.csv","a+")
            grid_log.write(str(result[0][0])+","+str(result[0][1])+","+str(result[1][0])+","+str(result[1][1])+"\r\n")
            grid_log.close()
        #workers_executioTime_grid=time.time()-workers_executioTime_grid
        #workers_executioTime_bestTraining= time.time()
        best_par = sorted(grid_results, key=lambda x: x[1])[0]
    best_par = (0.5, 0)
    print("\n--------------------------------------------------")
    print("Best parameters: " + str(best_par))
    smallest_error = train(DATASET,ITERATIONS,best_par[0],best_par[1])
    print("Best performance: " + str(smallest_error))
    print("--------------------------------------------------")
    
    #workers_executioTime_bestTraining=time.time()-workers_executioTime_bestTraining
    #working_time.write(str(worker)+","+str(workers_executioTime_grid)+","+str(workers_executioTime_bestTraining)+","+str(workers_executioTime_grid+workers_executioTime_bestTraining)+"\r\n")
    #working_time.close()
    print("++++Worker "+str(worker)+" has terminated with running time: "+str(workers_executioTime_grid+workers_executioTime_bestTraining)+"++++\n")

    

****Worker: 8****


--------------------------------------------------
Best parameters: (0.5, 0)
Split train/test done. Train contains 1 elements, Test contains 0 elements
b: 0.25
[-0.0855991229370825, 0.08271223578137651, 0.17819532540544952, -0.011723621718133783, 0.0028908634400629397, -0.08755702754164972, -0.0729405436671034, -0.06563325596098812, -0.08081680569620409, -0.09283100663187886, -0.07420681613626252, 0.02851873344344984, -0.0780053231271851, -0.043727038988629945, -0.04752343598508661, 0.02154051784415609, -0.08027512843329017, 0.5202451136825065, 0.03772224357872716, -0.04196871664391589, 0.031278699800448596, -0.029539666573631475, -0.0725444158808804, -0.05324281014056021, -0.08219473356820056, -0.0748018533014259, -0.05696751118741593, -0.057951240305905184, -0.041678332476904466, -0.05630375989049854, -0.040130464963906196, -0.03579911421235941, -0.04372531181573418, -0.03629984154690151, -0.04951146604779623, -0.06052597705787563, -0.08085511517083391, -0.0149574

Exception ignored in: <function JavaObject.__init__.<locals>.<lambda> at 0x115187730>
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1293, in <lambda>
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 625, in _garbage_collect_object
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 919, in garbage_collect_object
KeyboardInterrupt: 


b: 0.5291848139910746
[-0.053130989402829266, 0.051339111570275785, 0.11060503450145914, -0.007276799106070084, 0.0017943458943137348, -0.05434625195725372, -0.045273866362649476, -0.04073826585232177, -0.050162626668168536, -0.057619786983537466, -0.0460598359711166, 0.017701449178162458, -0.04841755212236293, -0.027141175813561257, -0.02949758230076243, 0.013370102239838666, -0.049826410035005285, 0.322913794832406, 0.023414026394936886, -0.026049793067356865, 0.0194145478435968, -0.01833513776584282, -0.045027991632981386, -0.03304757203451986, -0.05101790028141244, -0.046429172854858064, -0.03535947716927992, -0.03597007339475906, -0.025869552925041693, -0.034947489733981624, -0.024908798543357865, -0.0222203486739647, -0.027140103765137473, -0.02253114787134454, -0.030731543591136494, -0.03756820088006713, -0.050186405186227426, -0.009284011498039485, -0.028069662873004816, -0.028751190831309257, -0.01875920909157589, -0.02678009368416862, -0.03196127521252742, -0.0197582085999023

b: 0.9968142223072268
[-0.043128240813932836, 0.04167371230730955, 0.08978188843888274, -0.005906826651048394, 0.0014565319167448723, -0.04411471098300862, -0.03675034538246681, -0.0330686433617852, -0.040718719284549544, -0.046771951296302905, -0.03738834378843284, 0.014368871566143478, -0.03930218260613112, -0.02203142045004391, -0.02394419616865521, 0.010852969153929566, -0.040445800747106644, 0.2621201687038657, 0.019005965823983784, -0.021145507757140664, 0.015759452329108622, -0.01488325826056977, -0.036550760457164794, -0.026825844220861945, -0.041412974120914804, -0.03768814716575562, -0.02870249667005556, -0.029198138504482614, -0.020999200670646014, -0.028368072381677573, -0.020219323487818797, -0.01803701680227455, -0.022030550231681618, -0.018289303137989973, -0.024945844741069895, -0.030495393229312967, -0.04073802112072028, -0.007536149582515517, -0.02278510514408746, -0.023338324691427877, -0.015227491455999888, -0.021738317739082087, -0.025944059946545285, -0.0160384135

KeyboardInterrupt: 

In [4]:
grid = []
for i in range(1,5):
    for j in range(1,14,4):
        grid+=[(i*0.02,j*0.002)]
grid

[(0.02, 0.002),
 (0.02, 0.01),
 (0.02, 0.018000000000000002),
 (0.02, 0.026000000000000002),
 (0.04, 0.002),
 (0.04, 0.01),
 (0.04, 0.018000000000000002),
 (0.04, 0.026000000000000002),
 (0.06, 0.002),
 (0.06, 0.01),
 (0.06, 0.018000000000000002),
 (0.06, 0.026000000000000002),
 (0.08, 0.002),
 (0.08, 0.01),
 (0.08, 0.018000000000000002),
 (0.08, 0.026000000000000002)]